<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Qu'est-ce-qu'un-Jupyter-notebook-?" data-toc-modified-id="Qu'est-ce-qu'un-Jupyter-notebook-?-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Qu'est-ce qu'un Jupyter notebook ?</a></span></li><li><span><a href="#Qu'est-ce-que-Julia-?" data-toc-modified-id="Qu'est-ce-que-Julia-?-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Qu'est-ce que Julia ?</a></span></li><li><span><a href="#Qu'est-ce-que-JuMP-?" data-toc-modified-id="Qu'est-ce-que-JuMP-?-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Qu'est-ce que JuMP ?</a></span><ul class="toc-item"><li><span><a href="#Construction-d'un-premier-problème-linéaire" data-toc-modified-id="Construction-d'un-premier-problème-linéaire-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Construction d'un premier problème linéaire</a></span></li><li><span><a href="#Un-second-problème-linéaire" data-toc-modified-id="Un-second-problème-linéaire-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Un second problème linéaire</a></span></li><li><span><a href="#Déclarer-de-multiples-variables-/-contraintes-(Optionel)" data-toc-modified-id="Déclarer-de-multiples-variables-/-contraintes-(Optionel)-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Déclarer de multiples variables / contraintes (Optionel)</a></span></li></ul></li></ul></div>

# Une prise en main rapide de Jupyter / Julia / JuMP

## Qu'est-ce qu'un Jupyter notebook ?

Un Jupyter notebook est un document qui contient 
+ du texte 
  - que l'on peut formatter à l'aide de Markdown
  - qui peut contenir des maths à l'aide de $\LaTeX$
+ du code
  - avec lequel on peut intéragir en ligne
  
Un notebook est une succession de cellule, chacune pouvant être soit du code, soit du texte.
Quelques astuces :
+ double-clicker pour rentrer dans une cellule
+ M / Y pour changer le type de cellule
+ Ctrl-enter pour executer la cellule
+ shift-enter pour executer la cellule et passer à la suivante

Vous pouvez télécharger le fichier .ipynb via l'onglet "file" en haut à gauche. Vous pouvez aussi télécharger un pdf / html.

## Qu'est-ce que Julia ?

Julia est un langage de programmation, comparable à Python. C'est un langage récent, développé pour le calcul scientifique. 

Quelques éléments intéressant :
+ langage open-source
+ langage compilé "Just-in-time"
+ langage disposant d'un terminal (comme python)
+ ...

Faisons nos premiers pas avec Julia. Exécuter les cellules suivantes (shift-enter), n'hésitez pas à modifier pour prendre en main :

In [ ]:
1+1

In [ ]:
a = [0 5 10 15]
a[1]

In [ ]:
sum(a)

In [ ]:
sum(x^2 for x in a)

In [ ]:
length(a)

In [ ]:
exp.(a) .- a #to apply an operation or function componentwise just add .  

In [ ]:
@doc exp # to get documentation on a function

In [ ]:
for i = 1:5
    println("itération ",i)
end

In [ ]:
function factorielle(n)
    if n == 0
        return 1
    end
    res = 1
    for i=1:n
        res = res * i
    end
    return res
end

In [ ]:
factorielle(5)

In [ ]:
round(1.9453;sigdigits=2)

In [ ]:
rand(3)

Further info can be found [here](https://learnxinyminutes.com/docs/julia/) or in the [documentation](https://docs.julialang.org/en/v1/)



## Qu'est-ce que JuMP ?

JuMP est l'un des packages phare de Julia.

Il s'agit d'un package de modélisation, qui permet d'écrire un problème d'optimisation de manière simple puis de demander à un Solver de le résoudre.

Nous allons maintenant faire nos premiers pas avec JuMP.

Plus d'information sur http://www.juliaopt.org/JuMP.jl/v0.20.0/quickstart/ 

Nous allons maintenant dire que nous souhaitons utiliser ces packages (comparable à "from packet import *" en python)

In [1]:
using JuMP, GLPK


### Construction d'un premier problème linéaire

Nous souhaitons résoudre le problème linéaire suivant
$$ \begin{align*} 
\min_{x,y} \quad & 2x+3y \\
s.c. \quad & x+y \geq 1 \\
& x \geq 0, y\geq 0 \\
\end{align*}$$

Commençons par construire le problème

In [2]:
OPTIMIZER = GLPK.Optimizer              # On définit un optimizer
m = Model(with_optimizer(OPTIMIZER))       # On construit un problème d'optimisation

@variable(m,x>=0)                       # x est une variable réelle positive de m
@variable(m,y>=0)                       # y est une variable réelle positive de m

### Remarque les fonctions @variable / @objective / @constraint sont des fonctions spécifiques (des macros) 
# qui autorise de donner un argument comme 2*x+3*y sans qu'il soit évalué. 
# Ce n'est pas un comportement générique des fonctions julia.

@objective(m,Min, 2*x+3*y)              # l'objectif de m est de Minimiser 2*x+3*y

@constraint(m,x+y >= 1 )                # m a pour contrainte x+y <=1

x + y ≥ 1.0

On peut vérifier que m est bien ce que l'on souhaite

In [ ]:
print(m)

On peut également résoudre m

In [ ]:
optimize!(m)
println(termination_status(m))
println(primal_status(m))
println(dual_status(m))

Et si on souhaite connaître la valeur optimale du problème ou des solutions optimales on peut les avoir de la manière suivante

In [ ]:
println(JuMP.objective_value(m))
println("x = ",JuMP.value(x))
println("y = ",JuMP.value(y))

### Un second problème linéaire

Nous allons maintenant construire un problème linéaire plus complexe.
$$
\begin{align*}
\min_{x\in R^n} \quad & \sum_{i=1}^n c_i x_i \\
s.c. \quad & \sum_{i=1}^n x_i \geq n \\
&  -1 \leq x_i \leq 2 & \forall i
\end{align*}
$$

In [4]:
m2 = Model(with_optimizer(OPTIMIZER))
n = 10                                    # on choisit n = 10, mais vous pouvez le modifier
c = rand(n)                               # c est choisi ici de manière aléatoire                                              

@variable(m2, -1<= x[1:n] <= 2)           # x est une variable de m2 contenant n éléments x[1], x[2],...,x[n] tous compris entre -1 et 2

@objective(m2,Min, sum(c[i]*x[i] for i=1:n) )

@constraint(m2,sum(x[i] for i=1:n) >= n)
                        
print(m2)            

Min 0.3459510243667152 x[1] + 0.6461550613526932 x[2] + 0.8964939226339235 x[3] + 0.2790162408983783 x[4] + 0.7211218632586334 x[5] + 0.33052783128596275 x[6] + 0.19716023327420173 x[7] + 0.12050261160263442 x[8] + 0.378350062003316 x[9] + 0.9990171027862575 x[10]
Subject to
 x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] ≥ 10.0
 x[1] ≥ -1.0
 x[2] ≥ -1.0
 x[3] ≥ -1.0
 x[4] ≥ -1.0
 x[5] ≥ -1.0
 x[6] ≥ -1.0
 x[7] ≥ -1.0
 x[8] ≥ -1.0
 x[9] ≥ -1.0
 x[10] ≥ -1.0
 x[1] ≤ 2.0
 x[2] ≤ 2.0
 x[3] ≤ 2.0
 x[4] ≤ 2.0
 x[5] ≤ 2.0
 x[6] ≤ 2.0
 x[7] ≤ 2.0
 x[8] ≤ 2.0
 x[9] ≤ 2.0
 x[10] ≤ 2.0


In [ ]:
optimize!(m2)
println(termination_status(m2))

In [ ]:
value.(x)

Ajoutons maintenant une série de contraintes de la forme
$$ x_i + x_{i+1} \leq 1, \qquad \forall i \in 2, \dots, n-1$$

In [ ]:
for i = 1 : n-1
    @constraint(m2, x[i]+x[i+1] <= 2)
end

In [ ]:
optimize!(m2)
value.(x)

### Déclarer de multiples variables / contraintes (Optionel)

Pour terminer avec ce tutoriel nous présentons une autre manière de construire le problème précédent.

In [5]:
m3 = Model(with_optimizer(OPTIMIZER))                                     

@variable(m3, -1<= x[1:n] <= 2)          

@objective(m3,Min, sum(c[i]*x[i] for i=1:n) )

@constraints(m3,begin
        sum(x[i] for i=1:n) >= n
        [i in 1 : n-1], x[i]+x[i+1] <= 2
        end)
                        
print(m3)            

Min 0.3459510243667152 x[1] + 0.6461550613526932 x[2] + 0.8964939226339235 x[3] + 0.2790162408983783 x[4] + 0.7211218632586334 x[5] + 0.33052783128596275 x[6] + 0.19716023327420173 x[7] + 0.12050261160263442 x[8] + 0.378350062003316 x[9] + 0.9990171027862575 x[10]
Subject to
 x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] ≥ 10.0
 x[1] + x[2] ≤ 2.0
 x[2] + x[3] ≤ 2.0
 x[3] + x[4] ≤ 2.0
 x[4] + x[5] ≤ 2.0
 x[5] + x[6] ≤ 2.0
 x[6] + x[7] ≤ 2.0
 x[7] + x[8] ≤ 2.0
 x[8] + x[9] ≤ 2.0
 x[9] + x[10] ≤ 2.0
 x[1] ≥ -1.0
 x[2] ≥ -1.0
 x[3] ≥ -1.0
 x[4] ≥ -1.0
 x[5] ≥ -1.0
 x[6] ≥ -1.0
 x[7] ≥ -1.0
 x[8] ≥ -1.0
 x[9] ≥ -1.0
 x[10] ≥ -1.0
 x[1] ≤ 2.0
 x[2] ≤ 2.0
 x[3] ≤ 2.0
 x[4] ≤ 2.0
 x[5] ≤ 2.0
 x[6] ≤ 2.0
 x[7] ≤ 2.0
 x[8] ≤ 2.0
 x[9] ≤ 2.0
 x[10] ≤ 2.0


Bien évidemment il est également possible de déclarer plusieurs variables avec `@variables` (noter le `s` supplémentaire) en utilisant un bloc ̀̀`begin`-`end` et une déclaration de variable par ligne.

Tested Version :
- JuMP 0.20 / Julia 1.0.5
- JuMP 0.20 / Julia 1.2.0
- JuMP 0.20 / Julia 1.3.0-rc3